In [1]:
import sys
import pandas as pd
from lxml import etree
import re

print(sys.executable)

/opt/anaconda3/envs/pouille/bin/python


## Preprocessing

1. extration des articles à lier et des features utiles dans df
2. chargement du référentiel des communes (COG 2011) dans le df main_insee_commune

In [2]:
# Extraction des features de chaque article à lier dans un df

xml_file = '../../data/PO_t7/PO_t7.xml'
xml_tree = etree.parse(xml_file)

data = []

i = 0 # compter les articles

for article in xml_tree.xpath('//article'):
    i+=1
    # on ne retient pas les lieux hors France
    if article.find('./localisationpa') is not None:
        continue
    vedettes = article.xpath('./vedette/i')
    # on ne retient que la première vedette
    vedette = vedettes[0].text if vedettes else ''

    # on ne retient que le premier dpt de localisation
    localisationde = article.find('./localisationde')
    localisationde = localisationde.text if localisationde is not None else 'none'

    # on ne retient que le premier canton de localisation
    localisationca = article.find('./localisationca')
    if localisationca is not None:
        localisationca = re.search(
            r'<localisationca>(.*?)<\/localisationca>',
            etree.tostring(localisationca, encoding=str)
        ).group(1)
        '''
        cases:
        - 'c<sup>on</sup> de '
        - 'c<sup>on</sup> du '
        - 'c<sup>on</sup> d’'
        - 'c<sup>on</sup> de la '
        - 'c<sup>on</sup> de l’'
        - 'c<sup>on</sup> des '
        '''
        localisationca = re.sub(r"c<sup>on</sup> (de |d’|de la |du |de l’|des )", '', localisationca)
    else:
        localisationca = 'none'

    #  on ne retient que la première commune de localisation
    localisationco = article.find('./localisationco')
    if localisationco is not None:
        localisationco = re.search(
            r'<localisationco>(.*?)<\/localisationco>',
            etree.tostring(localisationco, encoding=str)
        ).group(1)
        '''
        cases:
        - 'c<sup>on</sup> de '
        - 'c<sup>on</sup> du '
        - 'c<sup>on</sup> d’'
        - 'c<sup>on</sup> de la '
        - 'c<sup>on</sup> de l’'
        - 'c<sup>on</sup> des '
        - 'c<sup>nes</sup> de '
        - 'c<sup>nes</sup> d’'
            '''
        localisationco = re.sub(r"c<sup>ne(s)?</sup> (de |d’|de la |du |de l’|des )", '', localisationco)

    else:
        localisationco = 'none'

    # Extraire le contenu textuel de la balise "article"
    article_text = re.sub(r'\W', ' ', article.xpath('string(.)'))

    # on insère tout dans un dictionnaire
    data.append({
        'old-id': article.get('old-id'),
        'vedette': vedette,
        'localisationde': localisationde,
        'localisationca': localisationca,
        'localisationco': localisationco,
        'reference': article_text  # Ajouter la colonne "reference"
    })

df = pd.DataFrame(data)

'''
article : nombre d’articles dans la source => `//article`
article_to_link : nombre d’articles à lier (en France) => `//article[not(localisationpa)]`
article_commune : nombre d’articles de type commune => `//article[not(localisationpa) and not(localisationco)]`
article_loc_com : nombre d’articles localisés dans une commune => `//article[not(localisationpa) and localisationco]`
'''
report = dict(
    article = i,
    article_to_link = len(df.index),
    article_commune = (df.localisationco == 'none').sum(),
    article_loc_com = (df.localisationco != 'none').sum()
)

# check
[print(k,':',v) for k, v in report.items()]
df.iloc[1500:1510, :]

article : 4220
article_to_link : 3428
article_commune : 2748
article_loc_com : 680


,old-id,vedette,localisationde,localisationca,localisationco,reference
1500,PO7-01910,Leugney,Doubs,Vercel,Brémondans,Leugney Doubs con de Vercel c...
1501,PO7-01911,Levier,Doubs,none,none,Levier Doubs arr de Pontarlier ...
1502,PO7-01912,Levoncourt,Haut-Rhin,Ferrette,none,Levoncourt en all Luffendorf Haut R...
1503,PO7-01913,Leymen,Haut-Rhin,Huningue,none,Leymen Haut Rhin con d Huningue ...
1504,PO7-01914,Leymenthal,none,none,none,Leymenthal doyenné Leintal de...
1505,PO7-01915,Leyrieu,Isère,Crémieu,none,Leyrieu Isère con de Crémieu ...
1506,PO7-01916,Leyris,Ardèche,Vallon,Lagorce,Leyris Ardèche con de Vallon ...
1507,PO7-01918,Lhopital,Ain,Bellegarde,none,Lhopital Ain con de Bellegarde ...
1508,PO7-01919,Lichoud,Isère,Bourgoin,Saint-Marcel,Lichoud le Isère con de Bourgoi...
1509,PO7-01922,Liesle,Doubs,Quingey,none,Liesle Doubs con de Quingey ...


In [3]:
# Ajout du dpt code au df
dpt_df = pd.read_csv('departements-region.csv')
dpt_dict = dict(zip(dpt_df['dep_name'], dpt_df['num_dep']))

# on ajoute notre colonne avec les numéros des départements qui matchent
df['dpt_code'] = df['localisationde'].apply(lambda x: dpt_dict.get(x, 'none'))

#on renomme et réordonne pour plus de clarté.
df = df.rename(columns={'old-id' : 'article_id', 'localisationca' : 'canton_code', 'localisationco_present' : 'nom_commune'})
df = df[['article_id', 'vedette', 'localisationde', 'dpt_code', 'canton_code', 'localisationco', 'reference']]

df.iloc[1500:1510, :]

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference
1500,PO7-01910,Leugney,Doubs,25,Vercel,Brémondans,Leugney Doubs con de Vercel c...
1501,PO7-01911,Levier,Doubs,25,none,none,Levier Doubs arr de Pontarlier ...
1502,PO7-01912,Levoncourt,Haut-Rhin,68,Ferrette,none,Levoncourt en all Luffendorf Haut R...
1503,PO7-01913,Leymen,Haut-Rhin,68,Huningue,none,Leymen Haut Rhin con d Huningue ...
1504,PO7-01914,Leymenthal,none,none,none,none,Leymenthal doyenné Leintal de...
1505,PO7-01915,Leyrieu,Isère,38,Crémieu,none,Leyrieu Isère con de Crémieu ...
1506,PO7-01916,Leyris,Ardèche,07,Vallon,Lagorce,Leyris Ardèche con de Vallon ...
1507,PO7-01918,Lhopital,Ain,01,Bellegarde,none,Lhopital Ain con de Bellegarde ...
1508,PO7-01919,Lichoud,Isère,38,Bourgoin,Saint-Marcel,Lichoud le Isère con de Bourgoi...
1509,PO7-01922,Liesle,Doubs,25,Quingey,none,Liesle Doubs con de Quingey ...


In [4]:
# Chargement du référentiel des communes
fields = ['insee_code', 'DEP_id', 'NCCENR']
main_insee_commune = pd.read_csv("main_insee_commune.tsv",
                                 delimiter='\t',
                                 usecols=fields,
                                 dtype={'insee_code': 'string'})
main_insee_commune['DEP_id'] = main_insee_commune['DEP_id'].apply(lambda x: x[4:])
main_insee_commune.head(5)

,insee_code,DEP_id,NCCENR
0,01001,01,Abergement-Clémenciat
1,01002,01,Abergement-de-Varey
2,01004,01,Ambérieu-en-Bugey
3,01005,01,Ambérieux-en-Dombes
4,01006,01,Ambléon


In [5]:
# Liste les départements présents pour mieux filtrer quand pas de localisationde
dpt_list = df['dpt_code'].unique().tolist()
dpt_list.remove("none")
dpt_list.sort()
print(dpt_list)

['01', '05', '07', '15', '25', '26', '30', '38', '39', '42', '43', '48', '51', '52', '54', '67', '68', '69', '70', '71', '73', '74', '88']


## Liage des articles de type commune

In [6]:
# dataframe à charger
linked_places_df = pd.DataFrame()

In [7]:
# Extraction des articles de type commune (ceux qui n’ont pas de commune de localisation)

communes_df = df[df['localisationco'] == 'none']
communes_df.iloc[1500:1510, :]
#communes_df.shape

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference
1846,PO7-02322,Morestel,Isère,38,none,none,Morestel Isère arr de la Tour du...
1847,PO7-02323,Morêtel,Isère,38,Goncelin,none,Morêtel Isère con de Goncelin ...
1848,PO7-02324,Morette,Isère,38,Tullins,none,Morette Isère con de Tullins ...
1849,PO7-02325,Morey,Haute-Saône,70,Vitrey,none,Morey Haute Saône con de Vitrey ...
1851,PO7-02330,Mornans,Drôme,26,Bourdeaux,none,Mornans Drôme con de Bourdeaux ...
1852,PO7-02332,Morschwiller-le-Bas,Haut-Rhin,68,Mulhouse,none,Morschwiller le Bas en all Niedermorsch...
1853,PO7-02333,Morteau,Doubs,25,none,none,Morteau Doubs arr de Pontarlier ...
1854,PO7-02334,Morvillars,territoire de Belfort,none,Delle,none,Morvillars territoire de Belfort ...
1855,PO7-02335,Motey-Besuche,Haute-Saône,70,Pesmes,none,Motey Besuche Haute Saône con de ...
1856,PO7-02336,Motey-sur-Saône,Haute-Saône,70,Fresne-Saint-Mamès,none,Motey sur Saône Haute Saône con d...


### Exact match method

#### With dpt

In [8]:
# liage des communes avec dpt de localisation (dpt/exact match)
communes_exact_dpt_df = pd.merge(communes_df,
                                 main_insee_commune,
                                 how='inner',
                                 left_on=['vedette','dpt_code'],
                                 right_on = ['NCCENR','DEP_id'])
communes_exact_dpt_df = communes_exact_dpt_df.drop(columns=['DEP_id', 'NCCENR'], axis=1)

communes_exact_dpt_df['method'] = 'dpt_exact'

# actualiser le df des liages réalisés
linked_places_df = pd.concat([linked_places_df, communes_exact_dpt_df]).drop_duplicates()

In [9]:
# Tests
#communes_exact_dpt_df.head(5)
communes_exact_dpt_df.shape

(2151, 9)

In [10]:
# des doublons de liage exact match dans un même dpt ?
#communes_exact_dpt_df.article_id.duplicated().sum()
communes_exact_dpt_df.loc[communes_exact_dpt_df.duplicated(keep=False, subset=['article_id']), :]

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,insee_code,method
987,PO7-01954,Longeville,Doubs,25,Ornans,none,Longeville Doubs con d Ornans ...,25346,dpt_exact
988,PO7-01954,Longeville,Doubs,25,Ornans,none,Longeville Doubs con d Ornans ...,25347,dpt_exact


#### Without dpt

In [11]:
# liage des communes sans dpt de localisation (nodpt/exact match)
communes_exact_nodpt_df = pd.merge(communes_df.loc[communes_df['dpt_code'] == 'none'],
                                   main_insee_commune[main_insee_commune.DEP_id.isin(dpt_list)],
                                   how='inner',
                                   left_on='vedette',
                                   right_on = 'NCCENR')
communes_exact_nodpt_df = communes_exact_nodpt_df.drop(columns=['DEP_id', 'NCCENR'], axis=1)

communes_exact_nodpt_df['method'] = 'nodpt_exact'

# actualiser le df des liages réalisés
linked_places_df = pd.concat([linked_places_df, communes_exact_nodpt_df]).drop_duplicates()

In [12]:
# Test
communes_exact_nodpt_df
#linked_places_df.shape
communes_exact_nodpt_df.agg(['nunique', 'count', 'size'])

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,insee_code,method
nunique,12,11,3,1,6,1,12,19,1
count,21,21,21,21,21,21,21,21,21
size,21,21,21,21,21,21,21,21,21


In [13]:
communes_exact_nodpt_df.loc[communes_exact_nodpt_df.duplicated(keep=False,subset=['article_id']), :]

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,insee_code,method
2,PO7-00818,Châtenois,territoire de Belfort,none,Belfort,none,Châtenois territoire de Belfort c...,39121,nodpt_exact
3,PO7-00818,Châtenois,territoire de Belfort,none,Belfort,none,Châtenois territoire de Belfort c...,67073,nodpt_exact
4,PO7-00818,Châtenois,territoire de Belfort,none,Belfort,none,Châtenois territoire de Belfort c...,70141,nodpt_exact
5,PO7-00818,Châtenois,territoire de Belfort,none,Belfort,none,Châtenois territoire de Belfort c...,88095,nodpt_exact
6,PO7-00842,Chaux,territoire de Belfort,none,Giromagny,none,Chaux territoire de Belfort con d...,25139,nodpt_exact
7,PO7-00842,Chaux,territoire de Belfort,none,Giromagny,none,Chaux territoire de Belfort con d...,71121,nodpt_exact
11,PO7-02220,Montagne,none,none,none,none,Montagne archiprêtré Montane ...,38245,nodpt_exact
12,PO7-02220,Montagne,none,none,none,none,Montagne archiprêtré Montane ...,70352,nodpt_exact
13,PO7-02221,Montagne,none,none,none,none,Montagne doyenné Montana 66 b...,38245,nodpt_exact
14,PO7-02221,Montagne,none,none,none,none,Montagne doyenné Montana 66 b...,70352,nodpt_exact


### Fuzzy method

#### With dpt

In [14]:
import difflib

# Créer un dictionnaire de correspondance entre les NCCENR et les codes INSEE
nccenr_to_insee = main_insee_commune.set_index('NCCENR')['insee_code'].to_dict()

# Lier les communes avec dpt de localisation (dpt/fuzzy match)
communes_fuzzy_dpt_df = communes_df[~communes_df.article_id.isin(linked_places_df['article_id'])][communes_df.dpt_code != 'none']
communes_fuzzy_dpt_df['insee_code'] = communes_fuzzy_dpt_df['vedette']
communes_fuzzy_dpt_df['insee_code'] = communes_fuzzy_dpt_df.apply(
    lambda x: [f"{nccenr} ({nccenr_to_insee.get(nccenr, '')})" for nccenr in difflib.get_close_matches(x['vedette'], main_insee_commune[main_insee_commune['DEP_id'] == x['dpt_code']]['NCCENR'], n=3)],
    axis=1
)

# Sortir les communes non liées
communes_fuzzy_dpt_df = communes_fuzzy_dpt_df[communes_fuzzy_dpt_df['insee_code'].map(lambda x: len(x)) > 0]

# Concaténer les codes INSEE avec les NCCENR dans la même cellule
communes_fuzzy_dpt_df['insee_code'] = communes_fuzzy_dpt_df['insee_code'].apply(lambda x: ', '.join(x))

communes_fuzzy_dpt_df['method'] = 'dpt_fuzzy'

# Actualiser le df des liages réalisés
linked_places_df = pd.concat([linked_places_df, communes_fuzzy_dpt_df]).drop_duplicates()



/var/folders/jj/dcr84y797818_4y88599z0500000gn/T/ipykernel_2375/1442772883.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  communes_fuzzy_dpt_df = communes_df[~communes_df.article_id.isin(linked_places_df['article_id'])][communes_df.dpt_code != 'none']


In [15]:
# Tests
communes_fuzzy_dpt_df.head(20)
#linked_places_df.shape
#linked_places_df
communes_fuzzy_dpt_df.agg(['nunique', 'count', 'size'])

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,insee_code,method
nunique,444,429,18,18,195,1,444,439,1
count,444,444,444,444,444,444,444,444,444
size,444,444,444,444,444,444,444,444,444


#### Without dpt

In [16]:
import difflib

# Créer un dictionnaire de correspondance entre les NCCENR et les codes INSEE
nccenr_to_insee = main_insee_commune.set_index('NCCENR')['insee_code'].to_dict()

# Lier les communes sans dpt de localisation (fuzzy match)
communes_fuzzy_nodpt_df = communes_df[~communes_df.article_id.isin(linked_places_df['article_id'])][communes_df.dpt_code == 'none']
communes_fuzzy_nodpt_df['insee_code'] = communes_fuzzy_nodpt_df['vedette']
communes_fuzzy_nodpt_df['insee_code'] = communes_fuzzy_nodpt_df.apply(
    lambda x: [f"{nccenr} ({nccenr_to_insee.get(nccenr, '')})" for nccenr in difflib.get_close_matches(x['vedette'], main_insee_commune['NCCENR'], n=3)],
    axis=1
)

# Sortir les communes non liées
communes_fuzzy_nodpt_df = communes_fuzzy_nodpt_df[communes_fuzzy_nodpt_df['insee_code'].map(lambda x: len(x)) > 0]

# Concaténer les codes INSEE avec les NCCENR dans la même cellule
communes_fuzzy_nodpt_df['insee_code'] = communes_fuzzy_nodpt_df['insee_code'].apply(lambda x: ', '.join(x))

communes_fuzzy_nodpt_df['method'] = 'nodpt_fuzzy'

# Actualiser le df des liages réalisés
linked_places_df = pd.concat([linked_places_df, communes_fuzzy_nodpt_df]).drop_duplicates()



/var/folders/jj/dcr84y797818_4y88599z0500000gn/T/ipykernel_2375/2094940778.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  communes_fuzzy_nodpt_df = communes_df[~communes_df.article_id.isin(linked_places_df['article_id'])][communes_df.dpt_code == 'none']


In [17]:
# Tests
communes_fuzzy_nodpt_df
#communes_fuzzy_nodpt_df.shape
#linked_places_df.shape
communes_fuzzy_nodpt_df.agg(['nunique', 'count', 'size'])

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,insee_code,method
nunique,99,99,3,1,11,1,99,99,1
count,99,99,99,99,99,99,99,99,99
size,99,99,99,99,99,99,99,99,99


## Liage des communes de localisation

In [18]:
# Extraction des articles appartenant à une commune (ceux ont une commune de localisation)

pas_communes_df = df[df['localisationco'] != 'none']
pas_communes_df

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference
0,PO7-00001,Abbaye,Drôme,26,Die,Die,Abbaye l Drôme con de Die ...
1,PO7-00002,Abbaye,Isère,38,Roybon,Marnans,Abbaye l Isère con de Roybon ...
2,PO7-00003,Abbaye,Haute-Savoie,74,Biot,Saint-Jean-d’Aulph,Abbaye l Haute Savoie con du B...
3,PO7-00005,Abbaye-d’Acey,Jura,39,Gendrey,Ougney,Abbaye d Acey l Jura con de Ge...
4,PO7-00006,Abbaye-de-Grandvaux,Jura,39,Saint-Laurent,Rivière-Devant,Abbaye de Grandvaux l Jura con...
...,...,...,...,...,...,...,...
3401,PO7-04245,Weinbach,Haut-Rhin,68,Kaysersberg,Kientzheim,Weinbach Haut Rhin con de Kaysers...
3402,PO7-04247,Weisskirch,Haut-Rhin,68,Huningue,Leymen,Weisskirch Haut Rhin con d Huning...
3411,PO7-04259,Wintzfelden,Haut-Rhin,68,Rouffach,Soultzmatt,Wintzfelden Haut Rhin con de Rouf...
3416,PO7-04268,Wolfenheim,Haut-Rhin,68,Colmar,Sainte-Croix-en-Plaine,Wolfenheim Haut Rhin con de Colma...


### Exact match method

#### With dpt

In [19]:
# liage des communes avec localisation (dpt/exact match)
pas_communes_exact_dpt_df = pd.merge(pas_communes_df,
                                 main_insee_commune,
                                 how='inner',
                                 left_on=['localisationco','dpt_code'],
                                 right_on = ['NCCENR','DEP_id'])
pas_communes_exact_dpt_df = pas_communes_exact_dpt_df.drop(columns=['DEP_id', 'NCCENR'], axis=1)

pas_communes_exact_dpt_df['method'] = 'dpt_exact'

# actualiser le df des liages réalisés
linked_places_df = pd.concat([linked_places_df, pas_communes_exact_dpt_df]).drop_duplicates()

In [20]:
# Tests
pas_communes_exact_dpt_df.head(5)
#pas_communes_exact_dpt_df.shape
pas_communes_exact_dpt_df.agg(['nunique', 'count', 'size'])

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,insee_code,method
nunique,519,466,16,16,190,409,519,411,1
count,519,519,519,519,519,519,519,519,519
size,519,519,519,519,519,519,519,519,519


#### Without dpt

In [21]:
# liage des communes sans dpt de localisation (nodpt/exact match)
pas_communes_exact_nodpt_df = pd.merge(pas_communes_df.loc[pas_communes_df['dpt_code'] == 'none'],
                                   main_insee_commune[main_insee_commune.DEP_id.isin(dpt_list)],
                                   how='inner',
                                   left_on='localisationco',
                                   right_on = 'NCCENR')
pas_communes_exact_nodpt_df = pas_communes_exact_nodpt_df.drop(columns=['DEP_id', 'NCCENR'], axis=1)

pas_communes_exact_nodpt_df['method'] = 'nodpt_exact'

# actualiser le df des liages réalisés
linked_places_df = pd.concat([linked_places_df, pas_communes_exact_nodpt_df]).drop_duplicates()

In [22]:
# Test
pas_communes_exact_nodpt_df
#linked_places_df.shape
pas_communes_exact_nodpt_df.agg(['nunique', 'count', 'size'])

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,insee_code,method
nunique,2,2,1,1,1,1,2,1,1
count,2,2,2,2,2,2,2,2,2
size,2,2,2,2,2,2,2,2,2


### Fuzzy match method

#### With dpt

In [23]:
import difflib

# Créer un dictionnaire de correspondance entre les NCCENR et les codes INSEE
nccenr_to_insee = main_insee_commune.set_index('NCCENR')['insee_code'].to_dict()

# Lier les communes avec dpt de localisation (dpt/fuzzy match)
pas_communes_fuzzy_dpt_df = pas_communes_df[~pas_communes_df.article_id.isin(linked_places_df['article_id'])][pas_communes_df.dpt_code != 'none']
pas_communes_fuzzy_dpt_df['insee_code'] = pas_communes_fuzzy_dpt_df['localisationco']
pas_communes_fuzzy_dpt_df['insee_code'] = pas_communes_fuzzy_dpt_df.apply(
    lambda x: [f"{nccenr} ({nccenr_to_insee.get(nccenr, '')})" for nccenr in difflib.get_close_matches(x['localisationco'], main_insee_commune[main_insee_commune['DEP_id'] == x['dpt_code']]['NCCENR'], n=3)],
    axis=1
)

# Sortir les communes non liées
pas_communes_fuzzy_dpt_df = pas_communes_fuzzy_dpt_df[pas_communes_fuzzy_dpt_df['insee_code'].map(lambda x: len(x)) > 0]

# Concaténer les codes INSEE avec les NCCENR dans la même cellule
pas_communes_fuzzy_dpt_df['insee_code'] = pas_communes_fuzzy_dpt_df['insee_code'].apply(lambda x: ', '.join(x))

pas_communes_fuzzy_dpt_df['method'] = 'dpt_fuzzy'

# Actualiser le df des liages réalisés
linked_places_df = pd.concat([linked_places_df, pas_communes_fuzzy_dpt_df]).drop_duplicates()



/var/folders/jj/dcr84y797818_4y88599z0500000gn/T/ipykernel_2375/3826840478.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  pas_communes_fuzzy_dpt_df = pas_communes_df[~pas_communes_df.article_id.isin(linked_places_df['article_id'])][pas_communes_df.dpt_code != 'none']


In [24]:
# Tests
pas_communes_fuzzy_dpt_df
#linked_places_df.shape
#linked_places_df
pas_communes_fuzzy_dpt_df.agg(['nunique', 'count', 'size'])

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,insee_code,method
nunique,140,133,12,12,80,106,140,108,1
count,140,140,140,140,140,140,140,140,140
size,140,140,140,140,140,140,140,140,140


#### Without dpt

In [25]:
import difflib

# Créer un dictionnaire de correspondance entre les NCCENR et les codes INSEE
nccenr_to_insee = main_insee_commune.set_index('NCCENR')['insee_code'].to_dict()

# Lier les communes sans dpt de localisation (fuzzy match)
pas_communes_fuzzy_nodpt_df = pas_communes_df[~pas_communes_df.article_id.isin(linked_places_df['article_id'])][pas_communes_df.dpt_code == 'none']
pas_communes_fuzzy_nodpt_df['insee_code'] = pas_communes_fuzzy_nodpt_df['localisationco']
pas_communes_fuzzy_nodpt_df['insee_code'] = pas_communes_fuzzy_nodpt_df.apply(
    lambda x: [f"{nccenr} ({nccenr_to_insee.get(nccenr, '')})" for nccenr in difflib.get_close_matches(x['localisationco'], main_insee_commune['NCCENR'], n=3)],
    axis=1
)

# Sortir les communes non liées
pas_communes_fuzzy_nodpt_df = pas_communes_fuzzy_nodpt_df[pas_communes_fuzzy_nodpt_df['insee_code'].map(lambda x: len(x)) > 0]

# Concaténer les codes INSEE avec les NCCENR dans la même cellule
pas_communes_fuzzy_nodpt_df['insee_code'] = pas_communes_fuzzy_nodpt_df['insee_code'].apply(lambda x: ', '.join(x))

pas_communes_fuzzy_nodpt_df['method'] = 'nodpt_fuzzy'

# Actualiser le df des liages réalisés
linked_places_df = pd.concat([linked_places_df, pas_communes_fuzzy_nodpt_df]).drop_duplicates()

/var/folders/jj/dcr84y797818_4y88599z0500000gn/T/ipykernel_2375/2582167694.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  pas_communes_fuzzy_nodpt_df = pas_communes_df[~pas_communes_df.article_id.isin(linked_places_df['article_id'])][pas_communes_df.dpt_code == 'none']


In [26]:
# Tests
pas_communes_fuzzy_nodpt_df
#linked_places_df.shape
pas_communes_fuzzy_nodpt_df.agg(['nunique', 'count', 'size'])

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,insee_code,method
nunique,4,4,2,1,3,4,4,4,1
count,4,4,4,4,4,4,4,4,4
size,4,4,4,4,4,4,4,4,4


## Rapport et analyse

In [27]:
#on append les deux dataframes ensemble

new_rows_df = df[~df['article_id'].isin(linked_places_df['article_id'])]

if 'method' not in new_rows_df.columns:
    new_rows_df['method'] = 'nulle'

linked_places_df = linked_places_df.append(new_rows_df, ignore_index=True)


/var/folders/jj/dcr84y797818_4y88599z0500000gn/T/ipykernel_2375/170212847.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_rows_df['method'] = 'nulle'
/var/folders/jj/dcr84y797818_4y88599z0500000gn/T/ipykernel_2375/170212847.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  linked_places_df = linked_places_df.append(new_rows_df, ignore_index=True)


In [28]:
linked_places_df.agg(['nunique', 'count', 'size'])

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,insee_code,method
nunique,3428,3171,26,24,344,531,3428,2893,5
count,3438,3438,3438,3438,3438,3438,3438,3380,3438
size,3438,3438,3438,3438,3438,3438,3438,3438,3438


In [29]:
# des doublons?
nombre_de_doublons = linked_places_df.duplicated(subset=['article_id']).sum()
print(nombre_de_doublons)


10


In [30]:
linked_places_df.loc[linked_places_df.duplicated(keep=False,subset=['article_id']), :]

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,insee_code,method
987,PO7-01954,Longeville,Doubs,25,Ornans,none,Longeville Doubs con d Ornans ...,25346,dpt_exact
988,PO7-01954,Longeville,Doubs,25,Ornans,none,Longeville Doubs con d Ornans ...,25347,dpt_exact
2153,PO7-00818,Châtenois,territoire de Belfort,none,Belfort,none,Châtenois territoire de Belfort c...,39121,nodpt_exact
2154,PO7-00818,Châtenois,territoire de Belfort,none,Belfort,none,Châtenois territoire de Belfort c...,67073,nodpt_exact
2155,PO7-00818,Châtenois,territoire de Belfort,none,Belfort,none,Châtenois territoire de Belfort c...,70141,nodpt_exact
2156,PO7-00818,Châtenois,territoire de Belfort,none,Belfort,none,Châtenois territoire de Belfort c...,88095,nodpt_exact
2157,PO7-00842,Chaux,territoire de Belfort,none,Giromagny,none,Chaux territoire de Belfort con d...,25139,nodpt_exact
2158,PO7-00842,Chaux,territoire de Belfort,none,Giromagny,none,Chaux territoire de Belfort con d...,71121,nodpt_exact
2162,PO7-02220,Montagne,none,none,none,none,Montagne archiprêtré Montane ...,38245,nodpt_exact
2163,PO7-02220,Montagne,none,none,none,none,Montagne archiprêtré Montane ...,70352,nodpt_exact


In [31]:
method_counts = linked_places_df['method'].value_counts()
print(method_counts)

dpt_exact      2670
dpt_fuzzy       584
nodpt_fuzzy     103
nulle            58
nodpt_exact      23
Name: method, dtype: int64


In [32]:
vedette_list = pd.unique(linked_places_df['vedette'])
linked_places_df['sort_key'] = linked_places_df.apply(lambda x: x['localisationco'] if x['localisationco'] in vedette_list else x['vedette'], axis=1)
linked_places_df.sort_values(['method', 'sort_key', 'article_id'], inplace=True)
linked_places_df = linked_places_df.drop('sort_key', axis=1)
linked_places_df = linked_places_df[['article_id', 'vedette', 'localisationde', 'dpt_code', 'canton_code', 'localisationco', 'insee_code', 'method', 'reference']]
linked_places_df

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,insee_code,method,reference
2721,PO7-00005,Abbaye-d’Acey,Jura,39,Gendrey,Ougney,39398,dpt_exact,Abbaye d Acey l Jura con de Ge...
0,PO7-00009,Abbenans,Doubs,25,Rougemont,none,25003,dpt_exact,Abbenans Doubs con de Rougemont ...
1,PO7-00012,Abondance,Haute-Savoie,74,none,none,74001,dpt_exact,Abondance Haute Savoie arr de Th...
2,PO7-00013,Abrets,Isère,38,Pont-de-Beauvoisin,none,38001,dpt_exact,Abrets les Isère con du Pont de...
3,PO7-00014,Accolans,Doubs,25,l’Isle-sur-le-Doubs,none,25005,dpt_exact,Accolans Doubs con de l Isle sur ...
...,...,...,...,...,...,...,...,...,...
3436,PO7-03908,Tromarey-et-Chancevigney,Haute-Saône,70,Marnay,none,NaN,nulle,Tromarey et Chancevigney Haute Saône ...
3437,PO7-03989,Vauclare,Ardèche,07,Montpezat,Mazan,NaN,nulle,Vauclare Ardèche con de Montpezat...
3395,PO7-01311,Ésery,Haute-Savoie,74,Reignier,Esserts-Ésery,NaN,nulle,Ésery Haute Savoie con de Reignie...
3397,PO7-01332,Étoile,Drôme,26,Valence,none,NaN,nulle,Étoile ou Étoile sur Rhone Drôme ...


## Export

In [33]:
# Sauvegarder le dataframe fusionné
#linked_places_df.to_csv('../../utils/pouille_linking/out/liage_po7.csv', index=False)
